# Генератор (болталка)

In [1]:
# медицинскую вопросно-ответную систему, датасет https://huggingface.co/datasets/blinoff/medical_qa_ru_data.

In [2]:
!pip3 install transformers torch wikipedia torchvision torchaudio

In [3]:
!pip3 install torch==1.12.1 torchvision==0.13.1 torchaudio==0.12.1

In [4]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
#DEVICE = torch.device("cuda:0")

model_name_or_path = "radm/rugpt3medium-tathagata"
model_name_or_path = "sberbank-ai/rugpt3medium_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3medium_based_on_gpt2")
model = GPT2LMHeadModel.from_pretrained(model_name_or_path)#.to(DEVICE)

In [5]:
def respond_to_dialog(texts):
    prefix = '\nx:'
    for i, t in enumerate(texts):
        prefix += t
        prefix += '\nx:' if i % 2 == 1 else '\ny:'
    tokens = tokenizer(prefix, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    end_token_id = tokenizer.encode('\n')[0]
    size = tokens['input_ids'].shape[1]
    output = model.generate(
        **tokens, 
        eos_token_id=end_token_id,
        do_sample=True, 
        max_length=size+128, 
        repetition_penalty=3.2, 
        temperature=1,
        num_beams=3,
        length_penalty=0.01,
        pad_token_id=end_token_id
    )
    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    return result.strip()

#"Бот-всезнайка"

In [6]:
!pip install wikipedia

In [7]:
import wikipedia
import re
wikipedia.set_lang("ru")

In [8]:
# Чистим текст статьи из Wikipedia и ограничиваем его тысячей символов
def getwiki(s):
    try:
        ny = wikipedia.page(s)
        # Получаем первую тысячу символов
        wikitext=ny.content[:2000]
        # Разбиваем, считая точку разделителем
        wikimas=wikitext.split('.')
        # Отбрасываем всё после последней точки
        wikimas = wikimas[:-1]
        # Создаем пустую переменную для текста
        wikitext2 = ''
        # Проходимся по строкам, где нет знаков «равно» (то есть все, кроме заголовков)
        for x in wikimas:
            if not('==' in x):
                    # Если в строке осталось больше трех символов, добавляем ее
                    # к нашей переменной и возвращаем утерянные при разделении строк точки на место
                if(len((x.strip()))>3):
                   wikitext2=wikitext2+x+'.'
            else:
                break
        # Теперь при помощи регулярных выражений убираем разметку
        wikitext2=re.sub('\([^()]*\)', '', wikitext2)
        wikitext2=re.sub('\{[^\{\}]*\}', '', wikitext2)
        # Возвращаем текстовую строку
        return wikitext2
    # Обрабатываем исключение, которое мог вернуть модуль wikipedia при запросе
    except Exception as e:
        return 'В энциклопедии нет информации об этом'

In [11]:
print(getwiki('Петербург'))

Санкт-Петербу́рг  — второй по численности населения город России. Город федерального значения. Административный центр Северо-Западного федерального округа. Основан 16  мая 1703 года царём Петром I. В 1714—1728 и 1732—1918 годах — столица Российского государства.
Назван в честь святого Петра — небесного покровителя царя-основателя, но со временем стал всё больше ассоциироваться с именем самого Петра I. Город исторически и культурно связан с рождением Российской империи и вхождением России в современную историю в роли европейской великой державы.
Расположен на северо-западе страны на побережье Финского залива и в устье реки Невы. Граничит с Ленинградской областью, также имеет морские границы с Финляндией и Эстонией.
В Санкт-Петербурге находятся: конституционный суд РФ, геральдический совет при президенте РФ, полпредство Северо-Западного федерального округа, органы власти Ленинградской области, межпарламентская ассамблея СНГ. Также размещены: главное командование ВМФ РФ и штаб Западного в

In [12]:
print(getwiki('Похвастаться нечем'))

В энциклопедии нет информации об этом


In [11]:
print(getwiki('Создать чат бот'))

В энциклопедии нет информации об этом


# Bot

In [13]:
!pip install pytelegrambotapi

In [14]:
history_dialog = []

def babbler(text):
    global history_dialog

    history_dialog.append(text)
    history_dialog.append(respond_to_dialog(history_dialog[-10:]))
    if len(history_dialog) > 10: history_dialog = history_dialog[-10:]
    
    return history_dialog[-1]

In [ ]:
import telebot
from telebot import types

# Создаем экземпляр бота
bot = telebot.TeleBot("6731295745:AAGO9xGZjfMemAe3ui8xwADSbUxtqvhaVJg")

# Функция, обрабатывающая команду /start
@bot.message_handler(commands=['start'])
def start(message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("Привет")
    markup.add(btn1)
    bot.send_message(message.chat.id, "👋 Привет! Я твой бот-помошник!", reply_markup=markup)

Flag = 0
@bot.callback_query_handler(func=lambda call: True)
def callback_worker(call):
    global Flag
    # Если нажали на одну из кнопок
    if call.data.lower() == 'поболтаем': 
        Flag = 1
        bot.send_message(call.message.chat.id, 'О чём ты хочешь побалтать?')
    if call.data.lower() == 'бот-всезнайка': 
        Flag = 2
        bot.send_message(call.message.chat.id, 'О чём тебе расказать?')


# Получение сообщений от пользователя
@bot.message_handler(content_types=["text"])
def handle_text(message):
    global Flag
    
    if message.text.lower() == 'привет': Flag = 0
    

    if Flag == 0:# or message.text == 'Назад':
        # Готовим кнопки
        keyboard = types.InlineKeyboardMarkup()
        # И добавляем кнопку на экран
        key_oven = types.InlineKeyboardButton(text='Поболтаем', callback_data='поболтаем')
        # И добавляем кнопку на экран
        keyboard.add(key_oven)
        key_telec = types.InlineKeyboardButton(text='Бот-всезнайка', callback_data='бот-всезнайка')
        keyboard.add(key_telec)
        # Показываем все кнопки сразу и пишем сообщение о выборе
        bot.send_message(message.chat.id, text='Сделайте выбор', reply_markup=keyboard)
    
    elif Flag == 1:
        bot.send_message(message.chat.id, babbler(message.text))
    
    elif Flag == 2:
        bot.send_message(message.chat.id, getwiki(message.text))


# Запускаем бота
bot.polling(none_stop=True, interval=0)